In [ ]:
from math import log2

from z3 import *

Range tracking part of value tracking will be done with the following C structure

```c
struct wrange {
	u64 base;
	u64 length;
}
```

In [ ]:
class Wrange:
    SIZE = 64 # Working with 64-bit integers
    name: str
    base: BitVecRef
    length: BitVecRef

    def __init__(self, name, base=None, length=None):
        self.name = name
        self.base = BitVec(f'Wrange-{name}-base', bv=self.SIZE) if base is None else base
        assert(self.base.size() == self.SIZE)
        self.length = BitVec(f'Wrange-{name}-length', bv=self.SIZE) if length is None else length
        assert(self.length.size() == self.SIZE)

    def wellformed(self):
        return If(self.length == BitVecVal(2**64 - 1, bv=self.SIZE), self.base == 0, True)
        
    def reset(self):
        return And(self.base == BitVecVal(0, bv=self.SIZE), self.length == BitVecVal(-1, bv=self.SIZE))

    @property
    def end(self):
        return self.base + self.length
        
    @property
    def uwrapping(self):
        return ULT(self.end, self.base)
        
    @property
    def umin(self):
        return If(self.uwrapping, BitVecVal(0, bv=self.SIZE), self.base)

    @property
    def umax(self):
        end = self.base + self.length
        return If(self.uwrapping, BitVecVal(-1, bv=self.SIZE), end)
        
    def contains(self, val: BitVecRef):
        assert(val.size() == self.SIZE)
        nonwrapping_cond = And(ULE(self.umin, val), ULE(val, self.umax))
        wrapping_cond = Or(ULE(x, w1.end), ULE(w1.base, x))
        return If(self.uwrapping, wrapping_cond, nonwrapping_cond)

In [ ]:
x = BitVec('x', bv=64)
w1 = Wrange('w1', BitVecVal(1, bv=64), BitVecVal(2, bv=64))
prove(
    w1.contains(x) == Or(x == BitVecVal(1, bv=64), x == BitVecVal(2, bv=64), x == BitVecVal(3, bv=64))
)

x = BitVec('x', bv=64)
w1 = Wrange('w1', BitVecVal(-1, bv=64), BitVecVal(2, bv=64))
prove(
    w1.contains(x) == Or(x == BitVecVal(-1, bv=64), x == BitVecVal(0, bv=64), x == BitVecVal(1, bv=64))
)

## Addition

In [ ]:
def wrange_add(a: Wrange, b: Wrange):
    new_length = a.length + b.length
    too_wide = Or(ULT(new_length, a.length), ULT(new_length, b.length))
    new_base = If(too_wide, BitVecVal(0, a.SIZE), a.base + b.base)
    new_length = If(too_wide, BitVecVal(2**64-1, a.SIZE), a.length + b.length)
    return Wrange(f'{a.name} + {b.name}', new_base, new_length)

In [ ]:
x = BitVec('x', bv=64)
w = wrange_add(
    # {1, 2, 3}
    Wrange('w1', BitVecVal(1, bv=64), BitVecVal(2, bv=64)),
    # + {0}
    Wrange('w2', BitVecVal(0, bv=64), BitVecVal(0, bv=64)),
)   # = {1, 2, 3}
prove(               # 1 <= x <= 3
    w.contains(x) == And(BitVecVal(1, bv=64) <= x, x <= BitVecVal(3, bv=64)),
)

x = BitVec('x', bv=64)
w = wrange_add(
    # {-1}
    Wrange('w1', BitVecVal(-1, bv=64), BitVecVal(0, bv=64)),
    # + {0, 1, 2}
    Wrange('w2', BitVecVal(0, bv=64), BitVecVal(2, bv=64)),   
)   # = {-1, 0, 1}
prove(               # -1 <= x <= 1
    w.contains(x) == And(BitVecVal(-1, bv=64) <= x, x <= BitVecVal(1, bv=64)),
)

In [ ]:
w1 = Wrange('w1')
w2 = Wrange('w2')
result = wrange_add(w1, w2)
x = BitVec('x', bv=64)
y = BitVec('y', bv=64)
premise = And(
    w1.wellformed(),
    w2.wellformed(),
    w1.contains(x),
    w2.contains(y),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(x + y),
            result.wellformed(),
        ),
    )
)

## Arithmetic Negation

In [ ]:
def wrange_neg(a: Wrange):
    return Wrange(f'(-{a.name})', If(a.length == -1, 0, - a.base - a.length), a.length)

In [ ]:
x = BitVec('x', bv=64)
w = wrange_neg(
    # -{1, 2, 3}
    Wrange('w1', BitVecVal(0x1, bv=64), BitVecVal(0x2, bv=64)),
)   # = {-3, -2, -1}
prove(
    Implies(
        w.contains(x),
        And(-3 <= x, x <= -1),
    )
)

x = BitVec('x', bv=64)
w = wrange_neg(
    # -{-1}
    Wrange('w1', BitVecVal(-1, bv=64), BitVecVal(0x0, bv=64)),
)   # = { 1}
prove(
    Implies(
        w.contains(x),
        x == 1,
    )
)

In [ ]:
w1 = Wrange('w1')
result = wrange_neg(w1)
x = BitVec('x', bv=64)
premise = And(
    w1.wellformed(),
    w1.contains(x),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(-x),
            result.wellformed(),
        ),
    )
)

## Subtraction

In [ ]:
#def wrange_sub(a: Wrange, b: Wrange):
#    new_length = a.length + b.length
#    too_wide = Or(new_length < a.length, new_length < b.length)
#    new_base = If(too_wide, BitVecVal(0, a.SIZE), a.base - b.base)
#    new_length = If(too_wide, BitVecVal(-1, a.SIZE), a.length + b.length)
#    return Wrange(f'{a.name} - {b.name}', new_base, new_length)

def wrange_sub(a: Wrange, b: Wrange):
    # Be a bit lazy here, improve later
    w = wrange_add(a, wrange_neg(b))
    return Wrange(f'{a.name} - {b.name}', w.base, w.length)

In [ ]:
x = BitVec('x', bv=64)
w = wrange_sub(
    # {1, 2, 3}
    Wrange('w1', BitVecVal(0x1, bv=64), BitVecVal(0x2, bv=64)),
    # - {0}
    Wrange('w2', BitVecVal(0x0, bv=64), BitVecVal(0x0, bv=64)),
)   # = {1, 2, 3}
prove(
    Implies(
        w.contains(x),
        # 1 <= x <= 3
        And(ULE(1, x), ULE(x, 3)),
    )
)

x = BitVec('x', bv=64)
w = wrange_sub(
    # {-1}
    Wrange('w1', BitVecVal(-1, bv=64), BitVecVal(0x0, bv=64)),
    # - {0, 1, 2}
    Wrange('w2', BitVecVal(0x0, bv=64), BitVecVal(0x2, bv=64)),
)   # = {-3, -2, -1}
prove(
    Implies(
        w.contains(x),
        # -3 <= x <= -1
        And(-3 <= x, x <= -1),
    )
)

In [ ]:
w1 = Wrange('w1')
w2 = Wrange('w2')
result = wrange_sub(w1, w2)
x = BitVec('x', bv=64)
y = BitVec('y', bv=64)
premise = And(
    w1.wellformed(),
    w2.wellformed(),
    w1.contains(x),
    w2.contains(y),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(x - y),
            result.wellformed(),
        ),
    )
)

## Multiplication

In [ ]:
def wrange_mul(a: Wrange, b: Wrange):
    too_wide = Or(UGT(a.length, BitVecVal(0xffffffff, bv=a.SIZE)), UGT(b.length, BitVecVal(0xffffffff, bv=b.SIZE)))
    new_base = If(too_wide, BitVecVal(0, a.SIZE), a.base * b.base)
    new_length = If(too_wide, BitVecVal(-1, a.SIZE), a.length * b.length)
    return Wrange(f'{a.name} * {b.name}', new_base, new_length)

In [ ]:
x = BitVec('x', bv=64)
w = wrange_mul(
    # {1, 2, 3}
    Wrange('w1', BitVecVal(0x1, bv=64), BitVecVal(0x2, bv=64)),
    # * {0}
    Wrange('w2', BitVecVal(0x0, bv=64), BitVecVal(0x0, bv=64)),
)   # = {0}
prove(
    Implies(
        w.contains(x),
        x == 0,
    )
)

x = BitVec('x', bv=64)
w = wrange_mul(
    # {-1}
    Wrange('w1', BitVecVal(-1, bv=64), BitVecVal(0x0, bv=64)),
    # {0, 1, 2}
    Wrange('w2', BitVecVal(0x0, bv=64), BitVecVal(0x2, bv=64)),
)   # {-2, -1, 0}
prove(
    Implies(
        w.contains(x),
        # -2 <= x <= 0
        And(-2 <= x, x <= 0),
    )
)

In [ ]:
w1 = Wrange('w1')
w2 = Wrange('w2')
result = wrange_mul(w1, w2)
x = BitVec('x', bv=64)
y = BitVec('y', bv=64)
premise = And(
    w1.wellformed(),
    w2.wellformed(),
    w1.contains(x),
    w2.contains(y),
)

In [ ]:
prove(
    Implies(
        premise,
        And(
            result.contains(x * y),
            result.wellformed(),
        ),
    )
)

## Evaluation

In [ ]:
s = Optimize()
x = BitVec('x', bv=64)
w1 = Wrange('w1', BitVecVal(-1, bv=64), BitVecVal(2, bv=64))
s.minimize(x)
s.minimize(w1.base)
#s.minimize(w2.base)
s.minimize(w1.length)
#s.minimize(w2.length)
s.add(
    Not(w1.contains(x) == Or(x == BitVecVal(-1, bv=64), x == BitVecVal(0, bv=64), x == BitVecVal(1, bv=64))),
)
s.check()

In [ ]:
m = s.model()
m

In [ ]:
m.eval(x)

In [ ]:
m.eval(w1.base), m.eval(w1.length), m.eval(w1.end)

In [ ]:
#m.eval(w2.base), m.eval(w2.length), m.eval(w2.base + w2.length)

In [ ]:
#m.eval(w.base), m.eval(w.length), m.eval(w.base + w.length)

In [ ]:
m.eval(w1.contains(x))

In [ ]:
m.eval(w1.wellformed())#, m.eval(w2.wellformed()), m.eval(w.wellformed())

In [ ]:
# wrapping?
m.eval(w1.uwrapping)

In [ ]:
# non-wrapping cond AND
m.eval(ULE(w1.base, x)), m.eval(ULE(x, w1.end))

In [ ]:
# wrapping cond OR
m.eval(ULE(x, w1.end)), m.eval(ULE(w1.base, x))